In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import joblib

from dotenv import load_dotenv
from typing import Iterable, TypeVar
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)

In [2]:
# Loading environment
load_dotenv(verbose=False)

True

In [3]:
# Consider including the location of the dataset in you pc or any cloud storage service
train_df = pd.read_csv(os.getenv('DATA_TRAIN'))
test_df = pd.read_csv(os.getenv('DATA_TEST'))

### Data manual processing
---
Before using the more sophisticated `PyCaret` processing tools. Let us do some manual data <span style="color:blue; ont-family:Courier; background-color:#FCE856; border-radius:10px; padding:3px;">pre-pre-processing</span> 💡. The process is made up of the fllowing steps:

<input type="checkbox" checked> Data imputation, fixing and transformation

<input type="checkbox" checked> Feature engineering using `feature_engine`

<input type="checkbox" checked> Save final `train` and `test` datasets 👉🏾 *to step 2...*


#### Data imputation, fixing and transformation
---
Now we try to extract more value from data by doing some operations over it

In [4]:
# Based in my expertise as an Analyst, there're some fields I think aren't important for predicting the performance of the students. 
# Even because the information they provide is irrelevant or it's likely redundant, I chose to remove them.
# However, there are other fields that may or may not be reundant, so I'll do a correlation analysis to see if its goot to get rid of them

# Excluded fields: 
#   'cole_cod_dane_establecimiento', 'cole_cod_dane_sede', 'cole_codigo_icfes', 'cole_depto_ubicacion', 'cole_mcpio_ubicacion', 
#   'cole_nombre_establecimiento', 'cole_nombre_sede', 'estu_depto_presentacion', 'estu_depto_reside', 'estu_id', 'estu_mcpio_presentacion', 
#   'estu_mcpio_reside', 'estu_tipodocumento', 'periodo', 'cole_sede_principal', 'estu_cod_reside_depto', 'cole_naturaleza', 
#   'estu_cod_reside_mcpio', 'estu_nacionalidad', 'estu_pais_reside', 'estu_estadoinvestigacion', 'estu_estudiante'

relevant_columns = [
    'cole_area_ubicacion', 'cole_bilingue', 'cole_calendario', 'cole_caracter', 'cole_cod_depto_ubicacion', 'cole_cod_mcpio_ubicacion', 
    'estu_cod_depto_presentacion', 'estu_cod_mcpio_presentacion', 'cole_genero', 'cole_jornada', 'estu_fechanacimiento', 'estu_genero', 
    'estu_privado_libertad', 'fami_cuartoshogar', 'fami_educacionmadre', 'fami_educacionpadre', 'fami_estratovivienda', 'fami_personashogar', 
    'fami_tieneautomovil', 'fami_tienecomputador', 'fami_tieneinternet', 'fami_tienelavadora', 'target'
]

In [5]:
# Data imputation, fixing and trasformation functions
# [DISCLAIMER] The method used to impute <cole_bilingue> isn't formal and is based ONLY in a quick analysis of the data
mode_columns = [
    'cole_caracter', 'fami_cuartoshogar', 'fami_educacionmadre', 'fami_educacionpadre', 'fami_estratovivienda', 'fami_personashogar',
    'fami_tieneautomovil', 'fami_tienecomputador', 'fami_tieneinternet', 'fami_tienelavadora', 'estu_genero'
]

def data_mode_simple_imputation(df:pd.DataFrame, columns:Iterable[str]=mode_columns) -> pd.DataFrame:
    for col in columns:
        df.loc[:, col] = df.loc[:, col].fillna(value=df[col].mode().values[0])
    
    return df

def data_bilingue_imputation(df:pd.DataFrame, column:str='cole_bilingue') -> pd.DataFrame:
    df.loc[df[column].isna(), column] = df.apply(
        lambda frame: 'S' if (
            frame[column] in ('Estrato 4', 'Estrato 5', 'Estrato 6') and 
            (
                frame.fami_tienecomputador == 'Si' or
                frame.fami_tieneinternet == 'Si' or
                frame.fami_tienelavadora == 'Si'
            )
        ) else 'N', axis=1
    )

    return df

def data_fecha_nacimiento_calculation(df:pd.DataFrame, column:str='estu_fechanacimiento') -> pd.DataFrame:
    df.loc[:, column] = df[column].str.replace(pat=r'/000', repl='/200', regex=False)
    df.loc[:, column] = pd.to_datetime(df[column], format='mixed', errors='coerce')

    df.loc[:, column] = df[column].apply(
        lambda x: np.round(
            (pd.Timestamp('31/12/2023')-x)/pd.Timedelta(365, 'day'), 2
        )
    )

    df = df.rename(columns={column:'estu_edad'})
    df.loc[:, 'estu_edad'] = pd.to_numeric(df.estu_edad, errors='coerce')

    return df

def data_lower_transformation(df:pd.DataFrame) -> pd.DataFrame:
    string_columns = df.select_dtypes('object').columns.to_list()
    df.loc[:, string_columns] = df.loc[:, string_columns].map(lambda x: x.lower() if type(x) == str else x)
    df.loc[:, string_columns] = df.loc[:, string_columns].map(lambda x: 'si' if x == 's' else ('no' if x == 'n' else x)) 

    return df

def data_get_dummies(df:pd.DataFrame) -> pd.DataFrame:
    df = df.reset_index(drop=True)

    binary_columns = [column for column in df.columns if len(df[column].unique()) == 2 and column != 'target']
    non_binary_target_columns = list(set(df.columns)-set(binary_columns))

    df_1 = df.copy()[binary_columns]
    df_1 = pd.get_dummies(df_1, drop_first=True, dtype=int)

    df_2 = df.copy()[non_binary_target_columns]
    
    res = pd.concat([df_1, df_2], axis=1, ignore_index=False)

    return res


In [6]:
# Train data processing
train_df_relevant_columns = train_df[relevant_columns]
train_df_relevant_columns = train_df_relevant_columns[train_df_relevant_columns.estu_genero.notna()]

train_df_relevant_columns = (
    train_df_relevant_columns
        .pipe(data_mode_simple_imputation)
        .pipe(data_bilingue_imputation)
        .pipe(data_fecha_nacimiento_calculation)
        .pipe(data_lower_transformation)
)

# Test data processing
test_df_relevant_columns = test_df[['estu_id']+[elem for elem in relevant_columns if elem != 'target']]

test_df_relevant_columns = (
    test_df_relevant_columns
        .pipe(data_mode_simple_imputation)
        .pipe(data_bilingue_imputation)
        .pipe(data_fecha_nacimiento_calculation)
        .pipe(data_lower_transformation)
)


#### Feature engineering
---
After completing the initial data processing, we now perform some **Data Engineering** tasks to enhance their features. These techniques are primarily based on an initial **Data Encoding** process

In [7]:
train_df_relevant_columns = train_df_relevant_columns.pipe(data_get_dummies)
test_df_relevant_columns = test_df_relevant_columns.pipe(data_get_dummies)

In [8]:
# Columns without order
columns_no_order_1 = ['cole_calendario', 'cole_jornada', 'cole_genero', 'cole_caracter']
columns_no_order_2 = ['estu_cod_depto_presentacion', 'cole_cod_depto_ubicacion', 'estu_cod_mcpio_presentacion', 'cole_cod_mcpio_ubicacion']

In [9]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    # For training data
    cfenc_train = CountFrequencyEncoder(encoding_method='frequency')
    cfenc_train.fit(train_df_relevant_columns[columns_no_order_1])
    train_df_relevant_columns.loc[:, columns_no_order_1] = cfenc_train.transform(train_df_relevant_columns[columns_no_order_1])

    scaler_train = MinMaxScaler()
    scaler_train.fit(train_df_relevant_columns[columns_no_order_2])
    train_df_relevant_columns.loc[:, columns_no_order_2] = pd.DataFrame(
        data=scaler_train.transform(train_df_relevant_columns[columns_no_order_2]).astype('float'), columns=columns_no_order_2
    )

    # For test data
    cfenc_test = CountFrequencyEncoder(encoding_method='frequency')
    cfenc_test.fit(test_df_relevant_columns[columns_no_order_1])
    test_df_relevant_columns.loc[:, columns_no_order_1] = cfenc_test.transform(test_df_relevant_columns[columns_no_order_1])

    scaler_test = MinMaxScaler()
    scaler_test.fit(test_df_relevant_columns[columns_no_order_2])
    test_df_relevant_columns.loc[:, columns_no_order_2] = pd.DataFrame(
        data=scaler_test.transform(test_df_relevant_columns[columns_no_order_2]).astype('float'), columns=columns_no_order_2
    )

# In case you'd like to save the tranied scalers
# joblib.dump(cfenc_train, 'model/cfenc_train.gz')
# joblib.dump(scaler_train, 'model/scaler_train.gz')

# joblib.dump(cfenc_test, 'model/cfenc_test.gz')
# joblib.dump(scaler_test, 'model/scaler_test.gz');

In [10]:
train_columns_sorted = [
    'cole_area_ubicacion_urbano','cole_bilingue_si','cole_calendario','cole_caracter','cole_cod_depto_ubicacion','cole_genero',
    'cole_jornada','estu_cod_depto_presentacion','estu_edad','estu_genero_m','estu_privado_libertad_si','fami_cuartoshogar',
    'fami_educacionmadre','fami_educacionpadre','fami_estratovivienda','fami_personashogar','fami_tieneautomovil_si','fami_tienecomputador_si',
    'fami_tieneinternet_si','fami_tienelavadora_si','target'
]

train_df_relevant_columns = train_df_relevant_columns[train_columns_sorted]

test_columns_sorted = [
    'estu_id','cole_area_ubicacion_urbano','cole_bilingue_si','cole_calendario','cole_caracter','cole_cod_depto_ubicacion',
    'cole_genero','cole_jornada','estu_cod_depto_presentacion','estu_edad','estu_genero_m','estu_privado_libertad_si',
    'fami_cuartoshogar','fami_educacionmadre','fami_educacionpadre','fami_estratovivienda','fami_personashogar','fami_tieneautomovil_si',
    'fami_tienecomputador_si','fami_tieneinternet_si','fami_tienelavadora_si'
]

test_df_relevant_columns = test_df_relevant_columns[test_columns_sorted]


In [11]:
# Saving processed data
train_df_relevant_columns.to_csv('data/train_df.csv', sep=';', index=False)
test_df_relevant_columns.to_csv('data/test_df.csv', sep=';', index=False)